## POS tagging using modified Viterbi


### Data Preparation

In [66]:
#Importing libraries
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [67]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [68]:
print(nltk_data[:40])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [69]:
# Splitting into train and test
random.seed(1234)
train_set, test_set = train_test_split(nltk_data,test_size=0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:40])

3718
196
[[('Ports', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('Great', 'NOUN'), ('Lakes', 'NOUN'), ('and', 'CONJ'), ('Atlantic', 'NOUN'), ('Coast', 'NOUN'), ('can', 'VERB'), ('also', 'ADV'), ('relieve', 'VERB'), ('pressure', 'NOUN'), ('on', 'ADP'), ('New', 'NOUN'), ('Orleans', 'NOUN'), ('.', '.')], [('In', 'ADP'), ('September', 'NOUN'), (',', '.'), ('she', 'PRON'), ('pleaded', 'VERB'), ('guilty', 'ADJ'), ('and', 'CONJ'), ('paid', 'VERB'), ('a', 'DET'), ('$', '.'), ('500', 'NUM'), ('*U*', 'X'), ('fine', 'NOUN'), ('.', '.')], [('The', 'DET'), ('USX', 'NOUN'), ('citations', 'NOUN'), ('represented', 'VERB'), ('the', 'DET'), ('first', 'ADJ'), ('sizable', 'ADJ'), ('enforcement', 'NOUN'), ('action', 'NOUN'), ('taken', 'VERB'), ('*', 'X'), ('by', 'ADP'), ('OSHA', 'NOUN'), ('under', 'ADP'), ('Mr.', 'NOUN'), ('Scannell', 'NOUN'), ('.', '.')], [('He', 'PRON'), ('is', 'VERB'), ('just', 'ADV'), ('passing', 'VERB'), ('the', 'DET'), ('buck', 'NOUN'), ('to', 'PRT'), ('young', 'ADJ'), ('people', 'NOUN'

In [70]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95453

In [71]:
# vocabulary
tokens = [pair[0] for pair in train_tagged_words]
V = set(tokens)
print(len(V))

12037


In [72]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [73]:
print(T)

{'NUM', 'CONJ', 'PRON', 'ADV', 'ADJ', 'PRT', 'ADP', 'X', 'NOUN', '.', 'VERB', 'DET'}


### Build the vanilla Viterbi based POS tagger

In [74]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [75]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [76]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [77]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [78]:
tags_matrix

array([[1.81102365e-01, 1.39309512e-02, 1.51423377e-03, 2.72562075e-03,
        3.39188352e-02, 2.54391283e-02, 3.57359163e-02, 2.11992726e-01,
        3.53119314e-01, 1.19018778e-01, 1.81708056e-02, 3.33131431e-03],
       [4.09302339e-02, 4.65116289e-04, 5.90697676e-02, 5.53488359e-02,
        1.17209300e-01, 5.11627924e-03, 5.48837222e-02, 8.83720908e-03,
        3.42325568e-01, 3.53488363e-02, 1.60465121e-01, 1.19999997e-01],
       [7.30769243e-03, 4.99999989e-03, 7.30769243e-03, 3.50000001e-02,
        7.03846142e-02, 1.26923081e-02, 2.03846153e-02, 9.30769220e-02,
        2.09615380e-01, 4.19230759e-02, 4.87692297e-01, 9.61538497e-03],
       [3.19680311e-02, 7.32600736e-03, 1.53180156e-02, 7.55910724e-02,
        1.31202132e-01, 1.43190147e-02, 1.18881121e-01, 2.19780225e-02,
        3.23010311e-02, 1.37196139e-01, 3.45654339e-01, 6.82650656e-02],
       [2.05795188e-02, 1.69575233e-02, 6.58544595e-04, 4.28054016e-03,
        6.65130094e-02, 1.07013499e-02, 7.68850818e-02, 2.10

In [79]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))
tags_df

,NUM,CONJ,PRON,ADV,ADJ,PRT,ADP,X,NOUN,.,VERB,DET
NUM,0.181102,0.013931,0.001514,0.002726,0.033919,0.025439,0.035736,0.211993,0.353119,0.119019,0.018171,0.003331
CONJ,0.040930,0.000465,0.059070,0.055349,0.117209,0.005116,0.054884,0.008837,0.342326,0.035349,0.160465,0.120000
PRON,0.007308,0.005000,0.007308,0.035000,0.070385,0.012692,0.020385,0.093077,0.209615,0.041923,0.487692,0.009615
ADV,0.031968,0.007326,0.015318,0.075591,0.131202,0.014319,0.118881,0.021978,0.032301,0.137196,0.345654,0.068265
ADJ,0.020580,0.016958,0.000659,0.004281,0.066513,0.010701,0.076885,0.021073,0.699374,0.066184,0.011854,0.004939
PRT,0.052770,0.002294,0.017699,0.009833,0.085873,0.001967,0.019666,0.013766,0.247132,0.042937,0.404130,0.101934
ADP,0.063619,0.000853,0.069054,0.013960,0.106458,0.001279,0.017050,0.034101,0.319800,0.040068,0.008419,0.325341
X,0.001752,0.010196,0.055600,0.025490,0.016409,0.185439,0.145452,0.075036,0.062450,0.162817,0.204716,0.054644
NOUN,0.009261,0.042901,0.004685,0.016985,0.011896,0.043779,0.178228,0.029027,0.263992,0.239430,0.146748,0.013068
.,0.080256,0.057647,0.066655,0.052603,0.044587,0.002522,0.090975,0.027112,0.220231,0.093677,0.089083,0.174563


In [80]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [81]:
# Testing  Vanilla Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('Mr.', 'NOUN'),
  ('Neuberger', 'NOUN'),
  ('realized', 'VERB'),
  ('that', 'ADP'),
  (',', '.'),
  ('although', 'ADP'),
  ('of', 'ADP'),
  ('Italian', 'ADJ'),
  ('ancestry', 'NOUN'),
  (',', '.'),
  ('Mr.', 'NOUN'),
  ('Mariotta', 'NOUN'),
  ('still', 'ADV'),
  ('could', 'VERB'),
  ('qualify', 'VERB'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('minority', 'NOUN'),
  ('person', 'NOUN'),
  ('since', 'ADP'),
  ('he', 'PRON'),
  ('was', 'VERB'),
  ('born', 'VERB'),
  ('*-1', 'X'),
  ('in', 'ADP'),
  ('Puerto', 'NOUN'),
  ('Rico', 'NOUN'),
  ('.', '.')],
 [('Also', 'ADV'),
  (',', '.'),
  ('a', 'DET'),
  ('Communist', 'NOUN'),
  ('official', 'NOUN'),
  ('for', 'ADP'),
  ('the', 'DET'),
  ('first', 'ADJ'),
  ('time', 'NOUN'),
  ('said', 'VERB'),
  ('0', 'X'),
  ('the', 'DET'),
  ('future', 'NOUN'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('Berlin', 'NOUN'),
  ('Wall', 'NOUN'),
  ('could', 'VERB'),
  ('be', 'VERB'),
  ('open', 'ADJ'),
  ('to', 'PRT'),
  ('discussion', 'NOUN'),
  ('.', '.')],
 [('T

In [82]:
test_s=[test_set[i] for i in rndom]
test_s

[[('Mr.', 'NOUN'),
  ('Neuberger', 'NOUN'),
  ('realized', 'VERB'),
  ('that', 'ADP'),
  (',', '.'),
  ('although', 'ADP'),
  ('of', 'ADP'),
  ('Italian', 'ADJ'),
  ('ancestry', 'NOUN'),
  (',', '.'),
  ('Mr.', 'NOUN'),
  ('Mariotta', 'NOUN'),
  ('still', 'ADV'),
  ('could', 'VERB'),
  ('qualify', 'VERB'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('minority', 'NOUN'),
  ('person', 'NOUN'),
  ('since', 'ADP'),
  ('he', 'PRON'),
  ('was', 'VERB'),
  ('born', 'VERB'),
  ('*-1', 'X'),
  ('in', 'ADP'),
  ('Puerto', 'NOUN'),
  ('Rico', 'NOUN'),
  ('.', '.')],
 [('Also', 'ADV'),
  (',', '.'),
  ('a', 'DET'),
  ('Communist', 'NOUN'),
  ('official', 'NOUN'),
  ('for', 'ADP'),
  ('the', 'DET'),
  ('first', 'ADJ'),
  ('time', 'NOUN'),
  ('said', 'VERB'),
  ('0', 'X'),
  ('the', 'DET'),
  ('future', 'NOUN'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('Berlin', 'NOUN'),
  ('Wall', 'NOUN'),
  ('could', 'VERB'),
  ('be', 'VERB'),
  ('open', 'ADJ'),
  ('to', 'PRT'),
  ('discussion', 'NOUN'),
  ('.', '.')],
 [('T

In [83]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [84]:
print("Time taken in seconds: ", difference)
print(tagged_seq)

Time taken in seconds:  40.342307329177856
[('Mr.', 'NOUN'), ('Neuberger', 'NOUN'), ('realized', 'VERB'), ('that', 'ADP'), (',', '.'), ('although', 'ADP'), ('of', 'ADP'), ('Italian', 'ADJ'), ('ancestry', 'NUM'), (',', '.'), ('Mr.', 'NOUN'), ('Mariotta', 'NOUN'), ('still', 'ADV'), ('could', 'VERB'), ('qualify', 'VERB'), ('as', 'ADP'), ('a', 'DET'), ('minority', 'NOUN'), ('person', 'NOUN'), ('since', 'ADP'), ('he', 'PRON'), ('was', 'VERB'), ('born', 'NUM'), ('*-1', 'X'), ('in', 'ADP'), ('Puerto', 'NUM'), ('Rico', 'NUM'), ('.', '.'), ('Also', 'ADV'), (',', '.'), ('a', 'DET'), ('Communist', 'NOUN'), ('official', 'NOUN'), ('for', 'ADP'), ('the', 'DET'), ('first', 'ADJ'), ('time', 'NOUN'), ('said', 'VERB'), ('0', 'X'), ('the', 'DET'), ('future', 'ADJ'), ('of', 'ADP'), ('the', 'DET'), ('Berlin', 'NOUN'), ('Wall', 'NOUN'), ('could', 'VERB'), ('be', 'VERB'), ('open', 'ADJ'), ('to', 'PRT'), ('discussion', 'NOUN'), ('.', '.'), ('The', 'DET'), ('$', '.'), ('35.7', 'NUM'), ('million', 'NUM'), ('*U*

In [85]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

0.8951612903225806

### Vanilla Viterbi has 89% accuracy.

In [86]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('Italian', 'ADJ'), (('ancestry', 'NUM'), ('ancestry', 'NOUN'))],
 [('was', 'VERB'), (('born', 'NUM'), ('born', 'VERB'))],
 [('in', 'ADP'), (('Puerto', 'NUM'), ('Puerto', 'NOUN'))],
 [('Puerto', 'NOUN'), (('Rico', 'NUM'), ('Rico', 'NOUN'))],
 [('the', 'DET'), (('future', 'ADJ'), ('future', 'NOUN'))],
 [('loss', 'NOUN'), (('equals', 'NUM'), ('equals', 'VERB'))],
 [('or', 'CONJ'), (('force', 'NOUN'), ('force', 'VERB'))],
 [('into', 'ADP'), (('wrenching', 'NUM'), ('wrenching', 'ADJ'))],
 [(',', '.'), (('state-supervised', 'NUM'), ('state-supervised', 'ADJ'))],
 [('``', '.'), (('interventions', 'NUM'), ('interventions', 'NOUN'))],
 [('that', 'DET'), (('*T*-86', 'NUM'), ('*T*-86', 'X'))],
 [('mean', 'VERB'), (('firings', 'NUM'), ('firings', 'NOUN'))],
 [('is', 'VERB'), (('sufficient', 'NUM'), ('sufficient', 'ADJ'))]]

Testing the Vanilla Viterbi on the Test file provided.

In [87]:
## Testing
file_content = open("Test_sentences.txt").read()
words = word_tokenize(file_content)
words
start = time.time()
tagged_seq = Viterbi(words)
end = time.time()
difference = end-start
print(tagged_seq)
print(difference)

[('Android', 'NUM'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NUM'), ('.', '.'), ('Android', 'NUM'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NUM'), ('worldwide', 'NUM'), ('on', 'ADP'), ('smartphones', 'NUM'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NUM'), ('and', 'CONJ'), ('Twitter', 'NUM'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NUM'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NUM'), ("'s", 'PRT'), ('firehose', 'NUM'), ('.', '.'), ('Twitter', 'NUM'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NUM'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOU

### The problems in Vallina Viterbi Algorithm :
### Assigning the first tag "NUM" to the unknown words in the training corpus.

### Solve the problem of unknown words

#### 1) Solving the problem of assigning unknown words to the first tag "NUM" : changing the logic of Viterbi algorithm to call Unigram tagger backed by Rule based tagger when the emmission probability is 0.

In [94]:
# specify patterns for tagging
#Solution1:
patterns = [
    (r'(The|the|A|a|An|an)$', 'DET'), # articles or determinants
    (r'.*able$', 'ADJ'),              # adjectives
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*es$', 'VERB'),               # 3rd singular present
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                    # nouns
]
rule_based_tagger = nltk.RegexpTagger(patterns)

In [95]:
#Solution2
# lexicon backed up by the rule-based tagger. Since Lexicon (Unigram) Tagger seems to perform fairly well.
lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)

Implementing both solutions in Viterbi Algorithm

In [96]:
# Viterbi Algorithm - Modified
def Viterbi_Modified(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            
        pmax = max(p)
        #state_max = T[p.index(pmax)] 
        
        # Handling unknown words, by using lexicon_tagger which is backed up by rule_based_tagger
        if(pmax==0):
            state_max = lexicon_tagger.tag([word])[0][1]
        else:
            state_max = T[p.index(pmax)] 
            
        state.append(state_max)
    return list(zip(words, state))

In [100]:
Viterbi_Modified(["ancestry"])

[('ancestry', 'NOUN')]

In [101]:
# Tagging the test_set data sentences using Modified Viterbi Algorithm 
start = time.time()
tagged_seq_modified = Viterbi_Modified(test_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

Time taken in seconds:  41.523375034332275


#### Evaluating tagging accuracy

In [103]:
# Modified Viterbi Algorithm Accuracy
check_Modified = [i for i, j in zip(tagged_seq_modified, test_run_base) if i == j] 

accuracy_modified = len(check_Modified)/len(tagged_seq_modified)
accuracy_modified

0.9354838709677419

## The modified Viterbi gives an accuracy of 93% which is greater than vanilla viterbi algorithm (~89%)

In [104]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq_modified, test_run_base)) if j[0] != j[1]]

[(('born', 'NOUN'), ('born', 'VERB')),
 (('future', 'ADJ'), ('future', 'NOUN')),
 (('equals', 'NOUN'), ('equals', 'VERB')),
 (('force', 'NOUN'), ('force', 'VERB')),
 (('wrenching', 'VERB'), ('wrenching', 'ADJ')),
 (('state-supervised', 'VERB'), ('state-supervised', 'ADJ')),
 (('*T*-86', 'NOUN'), ('*T*-86', 'X')),
 (('sufficient', 'NOUN'), ('sufficient', 'ADJ'))]

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [105]:
# Checking the tagged words on the test data set
## Testing - Modified Viterbi
file_content = open("Test_sentences.txt").read()
words = word_tokenize(file_content)
words
start = time.time()
tagged_seq = Viterbi_Modified(words)
end = time.time()
difference = end-start
print(tagged_seq)
print(difference)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), (

#### Tagging on test file data with Vanilla Viterbi:

[('Android', 'NUM'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NUM'), ('.', '.'), ('Android', 'NUM'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NUM'), ('worldwide', 'NUM'), ('on', 'ADP'), ('smartphones', 'NUM'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NUM'), ('and', 'CONJ'), ('Twitter', 'NUM'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NUM'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NUM'), ("'s", 'PRT'), ('firehose', 'NUM'), ('.', '.'), ('Twitter', 'NUM'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NUM'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOUN'), ('post', 'NOUN'), ('and', 'CONJ'), ('interact', 'NUM'), ('with', 'ADP'), ('messages', 'NUM'), ('known', 'VERB'), ('as', 'ADP'), ('tweets', 'NUM'), ('.', '.'), ('Before', 'ADP'), ('entering', 'VERB'), ('politics', 'NOUN'), (',', '.'), ('Donald', 'NOUN'), ('Trump', 'NOUN'), ('was', 'VERB'), ('a', 'DET'), ('domineering', 'NUM'), ('businessman', 'NOUN'), ('and', 'CONJ'), ('a', 'DET'), ('television', 'NOUN'), ('personality', 'NUM'), ('.', '.'), ('The', 'DET'), ('2018', 'NUM'), ('FIFA', 'NUM'), ('World', 'NOUN'), ('Cup', 'NUM'), ('is', 'VERB'), ('the', 'DET'), ('21st', 'NUM'), ('FIFA', 'NUM'), ('World', 'NOUN'), ('Cup', 'NUM'), (',', '.'), ('an', 'DET'), ('international', 'ADJ'), ('football', 'NOUN'), ('tournament', 'NUM'), ('contested', 'NUM'), ('once', 'ADV'), ('every', 'DET'), ('four', 'NUM'), ('years', 'NOUN'), ('.', '.'), ('This', 'DET'), ('is', 'VERB'), ('the', 'DET'), ('first', 'ADJ'), ('World', 'NOUN'), ('Cup', 'NUM'), ('to', 'PRT'), ('be', 'VERB'), ('held', 'VERB'), ('in', 'ADP'), ('Eastern', 'NOUN'), ('Europe', 'NOUN'), ('and', 'CONJ'), ('the', 'DET'), ('11th', 'ADJ'), ('time', 'NOUN'), ('that', 'ADP'), ('it', 'PRON'), ('has', 'VERB'), ('been', 'VERB'), ('held', 'VERB'), ('in', 'ADP'), ('Europe', 'NOUN'), ('.', '.'), ('Show', 'NOUN'), ('me', 'PRON'), ('the', 'DET'), ('cheapest', 'ADJ'), ('round', 'NOUN'), ('trips', 'NUM'), ('from', 'ADP'), ('Dallas', 'NOUN'), ('to', 'PRT'), ('Atlanta', 'NOUN'), ('I', 'PRON'), ('would', 'VERB'), ('like', 'ADP'), ('to', 'PRT'), ('see', 'VERB'), ('flights', 'NOUN'), ('from', 'ADP'), ('Denver', 'NOUN'), ('to', 'PRT'), ('Philadelphia', 'NOUN'), ('.', '.'), ('Show', 'NOUN'), ('me', 'PRON'), ('the', 'DET'), ('price', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('flights', 'NOUN'), ('leaving', 'VERB'), ('Atlanta', 'NOUN'), ('at', 'ADP'), ('about', 'ADP'), ('3', 'NUM'), ('in', 'ADP'), ('the', 'DET'), ('afternoon', 'NOUN'), ('and', 'CONJ'), ('arriving', 'NUM'), ('in', 'ADP'), ('San', 'NOUN'), ('Francisco', 'NOUN'), ('.', '.'), ('NASA', 'NUM'), ('invited', 'NUM'), ('social', 'ADJ'), ('media', 'NOUN'), ('users', 'NOUN'), ('to', 'PRT'), ('experience', 'NOUN'), ('the', 'DET'), ('launch', 'NOUN'), ('of', 'ADP'), ('ICESAT-2', 'NUM'), ('Satellite', 'NUM'), ('.', '.')

In [109]:
# Find the list of word in "Test sentences.txt" file which are not present in Universal Treebank Corpus (tokens Vocabulary)
words_not_in_corpus = list(set(words) - set(tokens))
words_not_in_corpus

['interact',
 'OS',
 'worldwide',
 'ICESAT-2',
 'Twitter',
 'NASA',
 'firehose',
 'domineering',
 'Satellite',
 'Google',
 'Cup',
 'trips',
 '2011',
 'tweets',
 '2018',
 'Android',
 'smartphones',
 'FIFA',
 'online',
 'arriving',
 '2013',
 '21st',
 'invited',
 'tournament',
 '2015',
 'contested',
 'messages',
 'personality']

In [112]:
# Get the new tags for sentences in "Test sentences.txt" using the new modified Viterbi algorithm
test_sentences_tagged =[]
test_sentences_tagged = test_sentences_tagged + list( Viterbi_Modified(words) )
test_sentences_tagged

# Display the new tags for the unknown words which was present in "Test sentences.txt"
corrected_words=[tup for tup in test_sentences_tagged for word in words_not_in_corpus if tup[0]==word ]

In [121]:
corrected_words

[('Android', 'NOUN'),
 ('Google', 'NOUN'),
 ('Android', 'NOUN'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('smartphones', 'VERB'),
 ('2011', 'NUM'),
 ('2013', 'NUM'),
 ('Google', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('2015', 'NUM'),
 ('Google', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('firehose', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('online', 'NOUN'),
 ('interact', 'NOUN'),
 ('messages', 'VERB'),
 ('tweets', 'NOUN'),
 ('domineering', 'VERB'),
 ('personality', 'NOUN'),
 ('2018', 'NUM'),
 ('FIFA', 'NOUN'),
 ('Cup', 'NOUN'),
 ('21st', 'NOUN'),
 ('FIFA', 'NOUN'),
 ('Cup', 'NOUN'),
 ('tournament', 'NOUN'),
 ('contested', 'VERB'),
 ('Cup', 'NOUN'),
 ('trips', 'NOUN'),
 ('arriving', 'VERB'),
 ('NASA', 'NOUN'),
 ('invited', 'VERB'),
 ('ICESAT-2', 'NOUN'),
 ('Satellite', 'NOUN')]

In [113]:
# Get the new tags for sentences in "Test sentences.txt" using the new modified Viterbi algorithm
test_sentences_tagged_vanilla =[]
test_sentences_tagged_vanilla = test_sentences_tagged + list( Viterbi(words) )
test_sentences_tagged_vanilla

# Display the new tags for the unknown words which was present in "Test sentences.txt"
vanilla_words=[tup for tup in test_sentences_tagged_vanilla for word in words_not_in_corpus if tup[0]==word ]

In [122]:
vanilla_words

[('Android', 'NOUN'),
 ('Google', 'NOUN'),
 ('Android', 'NOUN'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('smartphones', 'VERB'),
 ('2011', 'NUM'),
 ('2013', 'NUM'),
 ('Google', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('2015', 'NUM'),
 ('Google', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('firehose', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('online', 'NOUN'),
 ('interact', 'NOUN'),
 ('messages', 'VERB'),
 ('tweets', 'NOUN'),
 ('domineering', 'VERB'),
 ('personality', 'NOUN'),
 ('2018', 'NUM'),
 ('FIFA', 'NOUN'),
 ('Cup', 'NOUN'),
 ('21st', 'NOUN'),
 ('FIFA', 'NOUN'),
 ('Cup', 'NOUN'),
 ('tournament', 'NOUN'),
 ('contested', 'VERB'),
 ('Cup', 'NOUN'),
 ('trips', 'NOUN'),
 ('arriving', 'VERB'),
 ('NASA', 'NOUN'),
 ('invited', 'VERB'),
 ('ICESAT-2', 'NOUN'),
 ('Satellite', 'NOUN'),
 ('Android', 'NUM'),
 ('Google', 'NUM'),
 ('Android', 'NUM'),
 ('OS', 'NUM'),
 ('worldwide', 'NUM'),
 ('smartphones', 'NUM'),
 ('2011', 'NUM'),
 ('2013', 'NUM'),
 ('Google', 'NUM'),
 ('Twitter', 'NUM'),
 ('2015', 'NUM'),
 ('Google', '

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

1) Words like Andriod, Google, Satellite, FIFA which were earlier marked as "NUM" are now marked as "NOUN"

2) Words like contested and invited which were earlier marked as "NUM" are now correctly marked as "VERB"

3) ICESAT-2 and personality which were earlier marked as "NUM" are now marked as "NOUN"